## Genereate and Import Data

Elimizde gerçek bir data olmadığı için işimize yarayacak satın alma geçmişi, ürün detayları ve müşteri etkileşimi gibi dataları kendimiz oluşturarak import ediyoruz. 
Data oluşturma işlemleri kalabalık görünmemesi için bunun için bir fonkisyon yazdım,
data_generate.py içerisinden build_data() fonkiyonunu çağırarak hızlıca yapabiliriz.

In [1]:
import pandas as pd
import data_generate # Data oluşturma işlemleri için hazırladığım kütüphane

In [2]:
# Aynı dataların oluşabilmesi için random_state=42 ayarlandı.
customer_interactions_df, purchase_history_df, product_details_df = data_generate.build_data(random_state=42)

In [3]:
customer_interactions_df.head(), purchase_history_df.head(), product_details_df.head()

(   customer_id  book_id  page_views  view_dates
 0           52       21           1  2023-07-28
 1           93        4           1  2023-09-12
 2           15       18           1  2023-06-18
 3           72       14           1  2023-08-12
 4           61       28           1  2023-06-16,
    customer_id  book_id purchase_dates
 0           88        1     2023-12-07
 1           11       21     2023-08-25
 2           62       14     2023-10-04
 3           72        1     2023-12-01
 4           15       10     2023-07-19,
    book_id                                        book_name   price  rating
 0        1             Reactive foreground capacity of Onto  157.47     3.0
 1        2          Universal national framework of Student  131.06     1.9
 2        3  Profit-focused content-based structure of Sound  463.90     2.0
 3        4      Optional composite collaboration of Surface  446.89     2.6
 4        5   Switchable bi-directional methodology of Range  344.81     2.5)

Literatürde tavsiye sistemleri için kullanılan bir çok yöntem var. 
Bunlardan bazıları kullanıcı tabanlı, bazıları ürün ya da içerik tabanlı, bazıları ise pattern ya da kural tabanlı olabilir.

Datamıza uygun olan ve en sık kullanılan yöntemlerden birkaçını deneyeceğim. Yöntemler arasındaki farklar ise kısaca aşağıda belirtilmiştir.

![Example Image](comparison_table.png)



## Apriori

In [4]:
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

Kitap satın almalarında birliktelik yakalayabilmek için satın almaları aylık olarak birleştirdim. 
Müşterilerin her ay içerisinde yaptığı farklı kitap alışverişleri aynı sepetteymiş gibi davranacağız. Bunun için müşteri numarası ve satın alma tarihinin Ay ve Yıl bilgisini birleştirerek aylik_sepet_id adında bir kolon oluşturdum.

In [5]:
purchase_history_df["aylik_sepet_id"] = (
    purchase_history_df["customer_id"].astype(str) 
    +
    pd.to_datetime(purchase_history_df["purchase_dates"]).dt.strftime("%Y%m")
)

In [6]:
# Adım 1: Her bir aylik_sepet_id içerisinde hangi kitabın bulunduğunu işaretlediğimiz matrisi oluşturuyoruz.
############################################

# Satırlar aylik_sepet_id leri temsil ederken sütunlar kitap_id leri temsil eder.

arl_matrix = purchase_history_df.pivot_table(
    index="aylik_sepet_id", 
    columns="book_id", 
    values={"customer_id": "count"}
)

arl_matrix_pv = arl_matrix.fillna(0).applymap(lambda x: 1 if x > 0 else 0)

# Drop the index name 'book_id'
arl_matrix_pv.columns = arl_matrix_pv.columns.droplevel()
arl_matrix_pv = arl_matrix_pv.rename_axis(None, axis='columns')

# Reset index if needed
arl_matrix_pv = arl_matrix_pv.reset_index(drop=True)

arl_matrix_pv

,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
0,1,0,1,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,1,1,1,0,1,0,0,0,1,...,1,1,1,0,0,1,0,1,0,0
2,0,0,0,0,0,0,0,0,1,0,...,1,1,0,1,0,0,1,1,0,0
3,0,0,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695,0,1,1,1,1,0,0,0,0,1,...,1,1,1,1,0,1,1,1,0,1
696,0,0,1,0,0,1,0,1,0,0,...,0,0,0,0,0,1,1,0,0,1
697,0,1,0,0,1,0,1,0,0,0,...,0,1,1,0,0,0,1,1,0,0
698,1,1,1,0,0,0,0,1,0,0,...,0,1,1,0,1,0,0,1,0,0


In [7]:
# Adım 2: Birliktelik kurallarını oluşturalım.
############################################

support_itemsets = apriori(arl_matrix_pv, min_support=0.02, use_colnames=True)    # burada tüm ürünlerin support değerlerini çıkardık.

# Şimdi birliktelik kuralını oluşturalım, ve rules isimli df e atalım

rules = association_rules(support_itemsets, metric="lift", min_threshold=1)  # burada da ARL için ürünlerin;
# antecedents, consequents, support, confidence ve lift derğerleri vardır.
# literatürde lift tercih edilir.
rules

/opt/homebrew/Caskroom/miniforge/base/lib/python3.9/site-packages/mlxtend/frequent_patterns/fpcommon.py:111: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(1),(2),0.355714,0.395714,0.158571,0.445783,1.126528,0.017810,1.090342
1,(2),(1),0.395714,0.355714,0.158571,0.400722,1.126528,0.017810,1.075103
2,(1),(3),0.355714,0.397143,0.162857,0.457831,1.152813,0.021588,1.111937
3,(3),(1),0.397143,0.355714,0.162857,0.410072,1.152813,0.021588,1.092143
4,(1),(6),0.355714,0.355714,0.138571,0.389558,1.095144,0.012039,1.055442
...,...,...,...,...,...,...,...,...,...
221217,(18),"(25, 27, 21, 22)",0.364286,0.037143,0.020000,0.054902,1.478130,0.006469,1.018791
221218,(21),"(25, 18, 27, 22)",0.391429,0.031429,0.020000,0.051095,1.625747,0.007698,1.020725
221219,(22),"(25, 18, 27, 21)",0.357143,0.022857,0.020000,0.056000,2.450000,0.011837,1.035109
221220,(25),"(18, 27, 21, 22)",0.365714,0.030000,0.020000,0.054688,1.822917,0.009029,1.026116


In [8]:
# Function to recommend products based on a given product
def recommend_products_apriori(product_id, num_recommendations=6):
    # Find association rules related to the given product ID
    related_rules = rules[rules['antecedents'] == frozenset({product_id})]
    
    # Sort related rules by lift in descending order
    related_rules_sorted = related_rules.sort_values(by='lift', ascending=False)
    
    # Extract consequent items from the sorted rules
    consequent_items = related_rules_sorted['consequents'].apply(lambda x: list(x))
    
    # Initialize recommended products list
    recommended_products = []
    
    # Extract recommended products from consequent items
    for consequent_item_list in consequent_items:
        for consequent_item in consequent_item_list:
            if consequent_item not in recommended_products:
                recommended_products.append(consequent_item)
            if len(recommended_products) >= num_recommendations:
                break
        
    # Return recommended products
    return recommended_products[:num_recommendations]

In [9]:
recommend_products_apriori(12,5)

[24, 21, 28, 5, 11]

In [10]:

product_id_sample = 12
book_name = product_details_df.loc[product_details_df["book_id"]==product_id_sample, "book_name"].to_list()
book_price = product_details_df.loc[product_details_df["book_id"]==product_id_sample, "price"].to_list()
recommended_products = recommend_products_apriori(product_id_sample,5)

print(f"Müşteri book_id'si {product_id_sample} olan ({book_name}) isimli kitabı alırsa:")
print("########################################")
print(f"Önerebileceğin kitapların isimleri ve fiyatları")

for product_id in recommended_products:  
    print("########################################")
    product_info = product_details_df[product_details_df['book_id'] == product_id]
    print(product_info[['book_id', "book_name", 'price']].values)

Müşteri book_id'si 12 olan (['Mandatory bandwidth-monitored encoding of Expert']) isimli kitabı alırsa:
########################################
Önerebileceğin kitapların isimleri ve fiyatları
########################################
[[24 'Optimized intangible functionalities of A' 307.13]]
########################################
[[21 'Managed methodical monitoring of Record' 353.93]]
########################################
[[28 'Persistent non-volatile structure of Dog' 133.45]]
########################################
[[5 'Switchable bi-directional methodology of Range' 344.81]]
########################################
[[11 'Reactive explicit Graphical User Interface of Say' 363.61]]
